<a href="https://colab.research.google.com/github/Dangvinh777/dataticket/blob/main/Final_exam_L03_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 📓 Library
!pip install --quiet backtesting ta emoji
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from IPython.display import display
import emoji
from backtesting import Backtest, Strategy
from ta.trend import sma_indicator
from ta.trend import ema_indicator
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from matplotlib import pyplot
print("Done !")

Done !


# ⌨ CODE

##  DATA

In [ ]:
#@title  1🖱️ 👨🏻‍💻 FULL-RUNNING
Name_data  = "FPT Corp"  #@param ["Giá vàng","FPT Corp"]
if Name_data == "Giá vàng":
  Link_data ="https://github.com/Dangvinh777/dataticket/raw/main/XAU_USD%20Historical%20Data.csv"
elif Name_data =="FPT Corp":
  Link_data = "https://github.com/Dangvinh777/dataticket/raw/main/FPT%20Historical%20Data.csv"
df=pd.read_csv(Link_data)
# Private Filter for data
df= df.drop('Vol.',axis=1)
# Drop NaN
df= df.dropna()
# Iterate over each column
for column in df.columns:
    if column != 'Date' and column != 'Change %':
        # Check if the column is of object type
        if df[column].dtype == 'object':
            # Remove commas from the column values
            df[column] = df[column].str.replace(',', '')

            # Convert the column to float
            df[column] = df[column].astype(float)

df["Date"] = pd.to_datetime(df['Date'])
df_sorted = df.sort_values('Date')
df_run= df_sorted.set_index('Date')
df_run

,Price,Open,High,Low,Change %
Date,,,,,
2013-06-07,1384.47,1413.72,1418.03,1377.86,-2.06%
2013-06-10,1386.90,1378.89,1389.06,1376.49,0.18%
2013-06-11,1379.60,1387.01,1387.12,1367.26,-0.53%
2013-06-12,1388.40,1379.50,1394.87,1374.35,0.64%
2013-06-13,1386.05,1388.26,1395.27,1374.70,-0.17%
...,...,...,...,...,...
2023-06-01,1977.88,1962.80,1983.27,1953.43,0.79%
2023-06-02,1947.63,1978.14,1983.52,1947.67,-1.53%
2023-06-05,1961.45,1947.59,1964.13,1938.14,0.71%


In [ ]:
#@title 🛠️ SETUP
Data_Link = "https://github.com/Dangvinh777/dataticket/raw/main/FPT%20Historical%20Data.csv" #@param {type:"string"}
#'https://github.com/Dangvinh777/dataticket/raw/main/XAU_USD%20Historical%20Data.csv'

Choose_Name_Data = False  #@param {type:"boolean"}
if Choose_Name_Data == True:
  Name_data  = "FPT Corp"  #@param ["Giá vàng","FPT Corp"]
  if Name_data == "Giá vàng":
    Data_Link ="https://github.com/Dangvinh777/dataticket/raw/main/XAU_USD%20Historical%20Data.csv"
  elif Name_data =="FPT Corp":
    Data_Link = "https://github.com/Dangvinh777/dataticket/raw/main/FPT%20Historical%20Data.csv"



df=pd.read_csv(Data_Link)
print(emoji.emojize("Done! Have a good day! :face_blowing_a_kiss: "))
print("--------------------------------------------------------------------")
Show_Data  = "Yes"  #@param ["No","Yes"]
if Show_Data == "Yes":
  display(df)



Done! Have a good day! 😘 
--------------------------------------------------------------------


,Date,Price,Open,High,Low,Vol.,Change %
0,06/14/2023,"84,200.0","84,300.0","84,800.0","84,100.0",658.60K,-0.24%
1,06/13/2023,"84,400.0","84,400.0","84,900.0","84,300.0",466.80K,-0.59%
2,06/12/2023,"84,900.0","84,500.0","84,900.0","83,600.0",519.70K,0.35%
3,06/09/2023,"84,600.0","84,300.0","84,700.0","83,700.0",680.20K,1.20%
4,06/08/2023,"83,600.0","84,700.0","84,900.0","83,500.0",1.24M,-1.65%
...,...,...,...,...,...,...,...
2590,01/21/2013,"6,988.5","7,007.0","7,044.1","6,951.4",454.17K,-0.53%
2591,01/18/2013,"7,025.5","7,007.0","7,062.6","6,988.5",1.39M,-0.53%
2592,01/17/2013,"7,062.6","7,136.7","7,173.8","7,062.6",1.12M,-1.30%
2593,01/16/2013,"7,155.3","7,155.3","7,266.5","7,081.1",1.41M,1.05%


### 🧼 DATA CLEANING

In [ ]:
# Private Filter for data
df= df.drop('Vol.',axis=1)
# Drop NaN
df= df.dropna()
df

,Date,Price,Open,High,Low,Change %
0,06/07/2023,"1,966.30","1,963.33","1,966.43","1,963.28",0.15%
1,06/06/2023,"1,963.44","1,961.36","1,966.42","1,954.53",0.10%
2,06/05/2023,"1,961.45","1,947.59","1,964.13","1,938.14",0.71%
3,06/02/2023,"1,947.63","1,978.14","1,983.52","1,947.67",-1.53%
4,06/01/2023,"1,977.88","1,962.80","1,983.27","1,953.43",0.79%
...,...,...,...,...,...,...
2599,06/13/2013,"1,386.05","1,388.26","1,395.27","1,374.70",-0.17%
2600,06/12/2013,"1,388.40","1,379.50","1,394.87","1,374.35",0.64%
2601,06/11/2013,"1,379.60","1,387.01","1,387.12","1,367.26",-0.53%
2602,06/10/2013,"1,386.90","1,378.89","1,389.06","1,376.49",0.18%


In [ ]:
# Iterate over each column
for column in df.columns:
    if column != 'Date' and column != 'Change %':
        # Check if the column is of object type
        if df[column].dtype == 'object':
            # Remove commas from the column values
            df[column] = df[column].str.replace(',', '')

            # Convert the column to float
            df[column] = df[column].astype(float)

In [ ]:
df["Date"] = pd.to_datetime(df['Date'])
df_sorted = df.sort_values('Date')
Set_index_method  = "Date"  #@param ["Date","Number"]
if Set_index_method == "Date":
  #set index theo Date
  df_v1= df_sorted.set_index('Date')
else:
  #set index từ 0
  df_v1 = df_sorted.reset_index(drop=True)
df_v1.head(5)

,Price,Open,High,Low,Change %
Date,,,,,
2013-06-07,1384.47,1413.72,1418.03,1377.86,-2.06%
2013-06-10,1386.90,1378.89,1389.06,1376.49,0.18%
2013-06-11,1379.60,1387.01,1387.12,1367.26,-0.53%
2013-06-12,1388.40,1379.50,1394.87,1374.35,0.64%
2013-06-13,1386.05,1388.26,1395.27,1374.70,-0.17%


##  📚 STRATEGY

### ▶ MOVING AVERAGE

In [ ]:
def __init__(self, symbol, SMA1, SMA2, EMA1, EMA2,start, end):
        self.symbol = symbol
        self.SMA1 = SMA1
        self.SMA2 = SMA2
        self.EMA1 = EMA1
        self.EMA2 = EMA2
        self.start = start
        self.end = end
        self.results = None
        self.get_data()

    def get_data(self):
        ''' Retrieves and prepares the data.
        '''
        raw = pd.read_csv('http://hilpisch.com/pyalgo_eikon_eod_data.csv',
                          index_col=0, parse_dates=True).dropna()
        raw = pd.DataFrame(raw[self.symbol])
        raw = raw.loc[self.start:self.end]
        raw.rename(columns={self.symbol: 'price'}, inplace=True)
        raw['return'] = np.log(raw / raw.shift(1))

        def Cal_SMA(self):
            raw['SMA1'] = raw['price'].rolling(self.SMA1).mean()
            raw['SMA2'] = raw['price'].rolling(self.SMA2).mean()
            self.data_SMA = raw
        def Cal_EMA(self):
            raw['EMA1'] = raw['price'].ewm(com=self.EMA1).mean()
            raw['EMA2'] = raw['price'].ewm(com=self.EMA2).mean()
            self.data_EMA = raw
    def set_parameters(self):
        ''' Updates  parameters and resp. time series.
        '''
        def set_SMA(self, SMA1=None, SMA2=None):

            if SMA1 is not None:
                self.SMA1 = SMA1
                self.data_SMA['SMA1'] = self.data['price'].rolling(self.SMA1).mean()
            if SMA2 is not None:
                self.SMA2 = SMA2
                self.data_SMA['SMA2'] = self.data['price'].rolling(self.SMA2).mean()

        def set_EMA(self, EMA1=None, EMA2=None):

            if EMA1 is not None:
                self.EMA1=EMA1
                self.data_EMA['EMA1']=self.data['price'].ewm(com=self.EMA1).mean()
            if EMA2 is not None:
                self.EMA2=EMA2
                self.data_EMA['EMA2']=self.data['price'].ewm(com=self.EMA2).mean()

    def run_strategy_SMA(self):
        ''' Backtests the trading strategy.
        '''
        data1 = self.data_SMA.copy().dropna()
        data1['position'] = np.where(data1['SMA1'] > data1['SMA2'], 1, -1)
        data1['strategy'] = data1['position'].shift(1) * data1['return']
        data1.dropna(inplace=True)
        data1['creturns'] = data1['return'].cumsum().apply(np.exp)
        data1['cstrategy'] = data1['strategy'].cumsum().apply(np.exp)
        self.results = data1
        # gross performance of the strategy
        aperf1 = data1['cstrategy'].iloc[-1]
        # out-/underperformance of strategy
        operf1 = aperf1 - data1['creturns'].iloc[-1]
        return round(aperf1, 2), round(operf1, 2)

    def run_strategy_EMA(self):

        data2 = self.data_EMA.copy().dropna()
        data2['position']= np.where(data2['EMA1'] > data2['EMA2'], 1, -1)
        data2['strategy']=data2['position'].shift(1)*data2['return']
        data2.dropna(inplace=True)
        data2['creturn']= data2['return'].cumsum().apply(np.exp)
        data2['cstrategy']=data2['strategy'].cumsum.apply(np.exp)

        self.results=data2
        aperf2 = data2['cstrategy'].iloc[-1]
        operf2 = aperf2 - data2['return'].iloc[-1]
        return round(aperf2,2) & round(operf2, 2)

IndentationError: ignored

#### ⚛ SIMPLE MA

#### ⚛ EXPONENTIAL MA

### ▶ BOLINGER BAND

### ▶ MACD

### ▶ ROC

### ▶ RSI

## 💻 MACHINE LEARNING

### ⚡ LINEAR REGRESSION

### ⚡ LOGISTIC REGRESSION

### ⚡ SUPPORT VECTOR MACHINE

In [ ]:
#@title Create Virtual Data for SVM
df= df_run
df
x =df['Price']
print(x)

Date
2013-06-07    1384.47
2013-06-10    1386.90
2013-06-11    1379.60
2013-06-12    1388.40
2013-06-13    1386.05
               ...   
2023-06-01    1977.88
2023-06-02    1947.63
2023-06-05    1961.45
2023-06-06    1963.44
2023-06-07    1966.30
Name: Price, Length: 2604, dtype: float64


In [ ]:
#@title SMA part
df['SMA 50'] = sma_indicator(close=df['Price'], window=10)
df['SMA 200'] = sma_indicator(close=df['Price'], window=50)
df.loc[
    (df['SMA 50'] > df['SMA 200'])
    & (df['SMA 50'].shift(1) <= df['SMA 200'].shift(1)), 'position_SMA'] = 1
df.loc[
    (df['SMA 50'] < df['SMA 200'])
    & (df['SMA 50'].shift(1) >= df['SMA 200'].shift(1)), 'position_SMA'] = -1

In [ ]:
df

,Price,Open,High,Low,Change %,SMA 50,SMA 200,position_SMA
Date,,,,,,,,
2013-06-07,1384.47,1413.72,1418.03,1377.86,-2.06%,NaN,NaN,NaN
2013-06-10,1386.90,1378.89,1389.06,1376.49,0.18%,NaN,NaN,NaN
2013-06-11,1379.60,1387.01,1387.12,1367.26,-0.53%,NaN,NaN,NaN
2013-06-12,1388.40,1379.50,1394.87,1374.35,0.64%,NaN,NaN,NaN
2013-06-13,1386.05,1388.26,1395.27,1374.70,-0.17%,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2023-06-01,1977.88,1962.80,1983.27,1953.43,0.79%,1960.656,1992.2396,NaN
2023-06-02,1947.63,1978.14,1983.52,1947.67,-1.53%,1957.763,1991.3222,NaN
2023-06-05,1961.45,1947.59,1964.13,1938.14,0.71%,1956.965,1991.0068,NaN


In [ ]:
''' df.to_excel('testdfSMA.xlsx') '''

In [ ]:
#@title EMA part
df['EMA 50'] = ema_indicator(close=df['Price'], window=20)
df['EMA 200'] = ema_indicator(close=df['Price'], window=100)
df.loc[
    (df['EMA 50'] > df['EMA 200'])
    & (df['EMA 50'].shift(1) <= df['EMA 200'].shift(1)), 'position_EMA'] = 1
df.loc[
    (df['EMA 50'] < df['EMA 200'])
    & (df['EMA 50'].shift(1) >= df['EMA 200'].shift(1)), 'position_EMA'] = -1


In [ ]:
df["Change"] = df["Change %"].str.replace('%', '')
df["Change"] = df["Change"].astype(float)
df = df.drop('Change %', axis = 1)
''' df.to_excel('testfull.xlsx') '''

" df.to_excel('testfull.xlsx') "

In [ ]:
df

,Price,Open,High,Low,SMA 50,SMA 200,position_SMA,Change
Date,,,,,,,,
2013-06-07,1384.47,1413.72,1418.03,1377.86,NaN,NaN,NaN,-2.06
2013-06-10,1386.90,1378.89,1389.06,1376.49,NaN,NaN,NaN,0.18
2013-06-11,1379.60,1387.01,1387.12,1367.26,NaN,NaN,NaN,-0.53
2013-06-12,1388.40,1379.50,1394.87,1374.35,NaN,NaN,NaN,0.64
2013-06-13,1386.05,1388.26,1395.27,1374.70,NaN,NaN,NaN,-0.17
...,...,...,...,...,...,...,...,...
2023-06-01,1977.88,1962.80,1983.27,1953.43,1960.656,1992.2396,NaN,0.79
2023-06-02,1947.63,1978.14,1983.52,1947.67,1957.763,1991.3222,NaN,-1.53
2023-06-05,1961.45,1947.59,1964.13,1938.14,1956.965,1991.0068,NaN,0.71


In [ ]:
df=df.dropna()
df

,Price,Open,High,Low,SMA 50,SMA 200,position_SMA,Change
Date,,,,,,,,
2013-09-20,1325.38,1366.18,1368.40,1324.86,1344.520,1345.2892,-1.0,-2.90
2014-01-16,1242.55,1241.49,1245.26,1236.55,1239.073,1238.0034,1.0,0.07
2014-04-01,1279.15,1283.79,1288.25,1277.44,1306.781,1308.0906,-1.0,-0.40
2014-06-23,1318.10,1313.70,1318.61,1309.91,1284.642,1284.2720,1.0,0.25
2014-08-08,1309.60,1313.50,1322.86,1306.16,1298.025,1298.0900,-1.0,-0.30
...,...,...,...,...,...,...,...,...
2022-08-23,1747.67,1736.40,1754.28,1730.52,1768.869,1771.0256,-1.0,0.69
2022-11-11,1770.69,1754.92,1773.17,1746.85,1684.400,1675.1474,1.0,0.90
2023-02-16,1837.44,1836.56,1845.45,1827.18,1858.966,1860.9834,-1.0,0.07


In [ ]:
# feature selection
def select_features(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=f_classif, k=6) #f_classif ky
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs


# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

NameError: ignored

## ⌛ BACK-TESTING